In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import time

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from module.lgbm_baseline import *

In [ ]:
# Train CA_1
# Fold: 0
# 3130397 1572498
# [100]	training's rmse: 2.6008	valid_1's rmse: 2.6206
# [200]	training's rmse: 2.49692	valid_1's rmse: 2.5229
# [300]	training's rmse: 2.45556	valid_1's rmse: 2.4921
# [400]	training's rmse: 2.43108	valid_1's rmse: 2.47597
# [500]	training's rmse: 2.41148	valid_1's rmse: 2.46585
# [600]	training's rmse: 2.39295	valid_1's rmse: 2.45826
# [700]	training's rmse: 2.37771	valid_1's rmse: 2.45305
# [800]	training's rmse: 2.36466	valid_1's rmse: 2.44889
# [900]	training's rmse: 2.35218	valid_1's rmse: 2.44613
# [1000]	training's rmse: 2.34095	valid_1's rmse: 2.44366
# [1100]	training's rmse: 2.3296	valid_1's rmse: 2.4418
# [1200]	training's rmse: 2.32009	valid_1's rmse: 2.44015
# [1300]	training's rmse: 2.31056	valid_1's rmse: 2.43845
# [1400]	training's rmse: 2.30141	valid_1's rmse: 2.43791

In [3]:
if not os.path.exists(TEMP_FEATURE_PKL):
    train_df = pd.read_csv(f'{ORI_CSV_PATH}/sales_train_validation.csv')
    prices_df = pd.read_csv(f'{ORI_CSV_PATH}/sell_prices.csv')
    calendar_df = pd.read_csv(f'{ORI_CSV_PATH}/calendar.csv')
    try:
        if not os.path.exists(BASE_PATH):
            os.makedirs(BASE_PATH)
    except OSError:
        print("Creation of the directory %s failed" % BASE_PATH)
    else:
        print("Successfully created the directory %s" % BASE_PATH)

    grid_df = extract_features(train_df, prices_df, calendar_df, target=TARGET, base_path=None, nan_mask_d=1913 - 28)
    grid_df['item_id'] = grid_df['item_id'].astype('category')
    grid_df['dept_id'] = grid_df['dept_id'].astype('category')
    grid_df['cat_id'] = grid_df['cat_id'].astype('category')
#     grid_df['item_id'] = grid_df['item_id'].apply(lambda x: int(x.split('_')[-1])).astype('category')
#     grid_df['dept_id'] = grid_df['dept_id'].apply(lambda x: int(x.split('_')[-1])).astype('category')
#     grid_df['cat_id'] = grid_df['cat_id'].replace({'HOBBIES': 0, 'HOUSEHOLD': 1, 'FOODS': 2}).astype('category')
#     for col in ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']:
#         grid_df[col] = grid_df[col].replace(
#             dict(zip(grid_df[col].unique(), np.arange(grid_df[col].unique().shape[0])))).astype('category')
    grid_df = reduce_mem_usage(grid_df)
    grid_df.to_pickle(TEMP_FEATURE_PKL)

Successfully created the directory ../cache/ver10
melt_train_df 38.025161027908325
extract_price_features 26.37391996383667
extract_calendar_features 60.48720049858093
extract_rolling_features 553.2958493232727
extract_encode_features 34.66676950454712
extract_sliding_shift_features 413.4159450531006
Mem. usage decreased to 7779.56 Mb (47.3% reduction)


In [7]:
M5_FEATURES = set(list(grid_df)) - set(['d', 'id', 'sales', 'state_id', 'store_id', 'wm_yr_wk'])

In [14]:
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'poisson',
#                     'objective': 'tweedie',
#                     'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': 1,
                    'seed':SEED,
                }

In [12]:
history_df.rmse_val.mean()

2.2639737169801104

In [15]:
history_df = train_evaluate_model(M5_FEATURES, TARGET, BASE_PATH)
print(history_df.rmse_val.mean())
history_df.to_pickle(f'{BASE_PATH}/history.pkl')

Train CA_1
Fold: 0


/home/wang/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 2.59529	valid_1's rmse: 2.61014
[200]	training's rmse: 2.49169	valid_1's rmse: 2.51224
[300]	training's rmse: 2.45051	valid_1's rmse: 2.48293
[400]	training's rmse: 2.42386	valid_1's rmse: 2.46778
[500]	training's rmse: 2.40384	valid_1's rmse: 2.45822
[600]	training's rmse: 2.38614	valid_1's rmse: 2.4514
[700]	training's rmse: 2.37223	valid_1's rmse: 2.44659
[800]	training's rmse: 2.35908	valid_1's rmse: 2.44385
[900]	training's rmse: 2.34691	valid_1's rmse: 2.43995
[1000]	training's rmse: 2.33618	valid_1's rmse: 2.43794
[1100]	training's rmse: 2.32559	valid_1's rmse: 2.43625
[1200]	training's rmse: 2.31589	valid_1's rmse: 2.43464
[1300]	training's rmse: 2.30706	valid_1's rmse: 2.43386
[1400]	training's rmse: 2.29809	valid_1's rmse: 2.43312
Fold: 1
[100]	training's rmse: 2.61314	valid_1's rmse: 2.55253
[200]	training's rmse: 2.50536	valid_1's rmse: 2.4638
[300]	training's rmse: 2.4627	valid_1's rmse: 2.43906
[400]	training's rmse: 2.43337	valid_1's rmse: 2.42655


/home/wang/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 2.05734	valid_1's rmse: 2.05389
[200]	training's rmse: 2.01571	valid_1's rmse: 2.02557
[300]	training's rmse: 1.99823	valid_1's rmse: 2.01792
[400]	training's rmse: 1.98548	valid_1's rmse: 2.01418
[500]	training's rmse: 1.97568	valid_1's rmse: 2.01223
[600]	training's rmse: 1.96626	valid_1's rmse: 2.01077
[700]	training's rmse: 1.95792	valid_1's rmse: 2.01012
[800]	training's rmse: 1.95046	valid_1's rmse: 2.00984
[900]	training's rmse: 1.94367	valid_1's rmse: 2.00983
[1000]	training's rmse: 1.93697	valid_1's rmse: 2.00985
[1100]	training's rmse: 1.93066	valid_1's rmse: 2.00996
[1200]	training's rmse: 1.92466	valid_1's rmse: 2.00998
[1300]	training's rmse: 1.91841	valid_1's rmse: 2.01008
[1400]	training's rmse: 1.91298	valid_1's rmse: 2.01061
Fold: 1
[100]	training's rmse: 2.05505	valid_1's rmse: 2.05756
[200]	training's rmse: 2.01435	valid_1's rmse: 2.02521
[300]	training's rmse: 1.99673	valid_1's rmse: 2.0177
[400]	training's rmse: 1.98377	valid_1's rmse: 2.0146

...

[100]	training's rmse: 1.72825	valid_1's rmse: 1.7065
[200]	training's rmse: 1.68288	valid_1's rmse: 1.67489
[300]	training's rmse: 1.6652	valid_1's rmse: 1.66852
[400]	training's rmse: 1.65277	valid_1's rmse: 1.666
[500]	training's rmse: 1.64272	valid_1's rmse: 1.66445
[600]	training's rmse: 1.63389	valid_1's rmse: 1.66324
[700]	training's rmse: 1.62614	valid_1's rmse: 1.66248
[800]	training's rmse: 1.61941	valid_1's rmse: 1.66195
[900]	training's rmse: 1.61295	valid_1's rmse: 1.66127
[1000]	training's rmse: 1.60729	valid_1's rmse: 1.66087
[1100]	training's rmse: 1.60159	valid_1's rmse: 1.66069
[1200]	training's rmse: 1.59635	valid_1's rmse: 1.66027
[1300]	training's rmse: 1.59134	valid_1's rmse: 1.66011
[1400]	training's rmse: 1.58673	valid_1's rmse: 1.66012
Fold: 1
[100]	training's rmse: 1.7175	valid_1's rmse: 1.73983
[200]	training's rmse: 1.67169	valid_1's rmse: 1.70746
[300]	training's rmse: 1.65572	valid_1's rmse: 1.69971
[400]	training's rmse: 1.6453	valid_1's rmse: 1.69667
[50

/home/wang/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 2.69649	valid_1's rmse: 2.66674
[200]	training's rmse: 2.61936	valid_1's rmse: 2.63547
[300]	training's rmse: 2.58596	valid_1's rmse: 2.6322
[400]	training's rmse: 2.5622	valid_1's rmse: 2.63319
[500]	training's rmse: 2.5439	valid_1's rmse: 2.63256
[600]	training's rmse: 2.52564	valid_1's rmse: 2.63394
[700]	training's rmse: 2.51031	valid_1's rmse: 2.63308
[800]	training's rmse: 2.49521	valid_1's rmse: 2.63345
[900]	training's rmse: 2.48196	valid_1's rmse: 2.63353
[1000]	training's rmse: 2.46943	valid_1's rmse: 2.63394
[1100]	training's rmse: 2.45726	valid_1's rmse: 2.63482
[1200]	training's rmse: 2.44589	valid_1's rmse: 2.63559
[1300]	training's rmse: 2.43438	valid_1's rmse: 2.63707
[1400]	training's rmse: 2.42348	valid_1's rmse: 2.63846
Fold: 1
[100]	training's rmse: 2.67767	valid_1's rmse: 2.72344
[200]	training's rmse: 2.60929	valid_1's rmse: 2.671
[300]	training's rmse: 2.58152	valid_1's rmse: 2.65981
[400]	training's rmse: 2.55768	valid_1's rmse: 2.65328
[5

/home/wang/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	training's rmse: 2.47305	valid_1's rmse: 2.43726
[200]	training's rmse: 2.32973	valid_1's rmse: 2.35495
[300]	training's rmse: 2.2735	valid_1's rmse: 2.33545
[400]	training's rmse: 2.23753	valid_1's rmse: 2.32713
[500]	training's rmse: 2.21273	valid_1's rmse: 2.32217
[600]	training's rmse: 2.19133	valid_1's rmse: 2.32018
[700]	training's rmse: 2.17503	valid_1's rmse: 2.31775
[800]	training's rmse: 2.15889	valid_1's rmse: 2.31665
[900]	training's rmse: 2.14628	valid_1's rmse: 2.31519
[1000]	training's rmse: 2.13434	valid_1's rmse: 2.31423
[1100]	training's rmse: 2.12307	valid_1's rmse: 2.31354
[1200]	training's rmse: 2.11161	valid_1's rmse: 2.3122
[1300]	training's rmse: 2.10114	valid_1's rmse: 2.3102
[1400]	training's rmse: 2.09171	valid_1's rmse: 2.30983
Fold: 1
[100]	training's rmse: 2.42763	valid_1's rmse: 2.61916
[200]	training's rmse: 2.30982	valid_1's rmse: 2.48042
[300]	training's rmse: 2.2645	valid_1's rmse: 2.43694
[400]	training's rmse: 2.23279	valid_1's rmse: 2.40765
[

In [16]:
final_all_preds = predict_test(M5_FEATURES, TARGET, BASE_PATH)
for col in [f'F{i}' for i in range(1, 29)]:
    print(col, final_all_preds[col].sum())
submission = pd.read_csv(f'{ORI_CSV_PATH}/sample_submission.csv')
submission = pd.concat([final_all_preds, submission[~submission['id'].isin(final_all_preds.id.unique().tolist())]],axis=0)
submission.to_csv(f'{BASE_PATH}/submission.csv', index=False)

FOLD0 Predict | Day:1
##########  2.69 min round |  2.69 min total |  37312.00 day sales |
FOLD0 Predict | Day:2
##########  2.71 min round |  5.40 min total |  35200.00 day sales |
FOLD0 Predict | Day:3
##########  2.71 min round |  8.12 min total |  34496.00 day sales |
FOLD0 Predict | Day:4
##########  2.72 min round |  10.83 min total |  35104.00 day sales |
FOLD0 Predict | Day:5
##########  2.70 min round |  13.54 min total |  41600.00 day sales |
FOLD0 Predict | Day:6
##########  2.69 min round |  16.23 min total |  49216.00 day sales |
FOLD0 Predict | Day:7
##########  2.68 min round |  18.91 min total |  52768.00 day sales |
FOLD0 Predict | Day:8
##########  2.68 min round |  21.59 min total |  43872.00 day sales |
FOLD0 Predict | Day:9
##########  2.71 min round |  24.30 min total |  44416.00 day sales |
FOLD0 Predict | Day:10
##########  2.72 min round |  27.02 min total |  38528.00 day sales |
FOLD0 Predict | Day:11
##########  2.69 min round |  29.71 min total |  39072.00 d